# Pairs Trading Algorithm for Forex Pairs

In [2]:
# Importing packages
import datetime as dt
from datetime import datetime as datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import numpy as np
from itertools import combinations
import statsmodels.api as sm
from statsmodels.tsa.api import VAR
import statsmodels.stats.diagnostic as dg
from binance import Client 
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.diagnostic import acorr_breusch_godfrey
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
import numpy as np
import pandas as pd
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.vecm import VECM, select_order, coint_johansen
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox
import numpy as np
import pandas as pd
import os
import sys

%load_ext autoreload
%autoreload 2


C:\Users\Jonas\AppData\Local\Temp\ipykernel_7348\2453397559.py:15: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Reading data

In [3]:
# set length of period for data
period = 5

# creating datetime objects for start and end of period
end_dt = dt.datetime(2024, 3, 16)
start_dt = end_dt - relativedelta(years = period)
model_end_dt = end_dt - relativedelta(years = 1)

# creating string objects for start and end of period
end = end_dt.strftime('%Y-%m-%d')
start = start_dt.strftime('%Y-%m-%d')
model_end = model_end_dt.strftime('%Y-%m-%d')

In [4]:
# checking for 20 most traded currencies
wikiurl = "https://en.wikipedia.org/wiki/Template:Most_traded_currencies"
table_class = "wikitable sortable jquery-tablesorter"
response = requests.get(wikiurl)
#print(response.status_code) # checks availability of table

soup = BeautifulSoup(response.text, 'html.parser')
wiki_currencytable = soup.find('table',{'class':"wikitable"})

wiki_currency = pd.read_html(str(wiki_currencytable))
wiki_currency = pd.DataFrame(wiki_currency[0])

tickers = list(wiki_currency.iloc[1:36,2])
tickers_yahoo = [s + '=X' for s in tickers]

C:\Users\Jonas\AppData\Local\Temp\ipykernel_7348\2269934189.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  wiki_currency = pd.read_html(str(wiki_currencytable))


In [5]:
# fetching gold price
gold = yf.download('GC=F', start, end, progress = False)

In [6]:
# fetching rates for the 20 tickers
d = []

for ticker in tickers_yahoo:
    full = yf.download(ticker, start, end, progress = False)
    d.append( (1 / full['Adj Close']) * gold['Adj Close'])

rates = pd.concat(d, axis = 1)
rates.columns = tickers

rates = rates.dropna()

# creating dataframe of returns and lagged returns
returns = rates.pct_change()
returns_lag = rates.pct_change().shift(1)
returns = returns.dropna()

# splitting into modelling and backtesting dataframes
returns_model = returns.loc[:model_end]
returns_test = returns.loc[model_end:]

# drop first day
rates = rates[1:]
returns = returns[1:]


In [7]:
c_tickers = ["BTC-USD", "ETH-USD", "LTC-USD", "XRP-USD", "LINK-USD"]

In [8]:
    c = []

    for ticker in c_tickers:
        full = yf.download(ticker, start, end, progress = True)
        c.append( (1 / full['Adj Close']) * gold['Adj Close'])

c_rates = pd.concat(c, axis = 1)
c_rates.columns = c_tickers

c_rates = c_rates.dropna()

# creating dataframe of returns and lagged returns
c_returns = c_rates.pct_change()
c_returns_lag = c_rates.pct_change().shift(1)
c_returns = c_returns.dropna()

# splitting into modelling and backtesting dataframes
c_returns_model = c_returns.loc[:model_end]
c_returns_test = c_returns.loc[model_end:]

# drop first day
c_rates = c_rates[1:]
c_returns = c_returns[1:]


rates = pd.concat([rates, c_rates], axis=1)
returns = pd.concat([returns, c_returns], axis=1)

rates = rates[1:]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [9]:
returns.to_csv('returns.csv', sep=',', index = True, encoding=None) 

In [10]:
rates.to_csv('rates.csv', sep=',', index = True, encoding=None) 

In [11]:
end_list = tickers + c_tickers


In [12]:
possible_pairs = list(combinations(end_list, 2)) # finds all possible pairs within the set of exchange rates

## Restrictions and definitions

In [13]:
significance_level = 0.05
beta_lower = -0.5
beta_upper = 2
root_restriction = 0.95
open_threshold = 1.25
close_threshold = 0.5
stoploss_threshold = 3.75

## Example

In [14]:
import warnings

# Filter out the specific ValueWarning related to date index frequency
warnings.filterwarnings("ignore", message="A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.")


In [15]:
returns = returns.dropna(axis=0) 
rates = rates.dropna(axis=0)


In [16]:
pairs = list(combinations(returns, 2))

In [17]:
significance_level = 0.05
beta_lower = -0.5
beta_upper = 2
root_restriction = 0.95
open_threshold = 1.25
close_threshold = 0.5
stoploss_threshold = 3.75

In [35]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller, coint
from statsmodels.tsa.vector_ar.vecm import select_order, VECM, coint_johansen
from statsmodels.tsa.vector_ar.var_model import VAR
from scipy.stats import chi2

# Assume 'returns' is your DataFrame with returns for each currency
# significance_level and perform_wb are previously defined

def wild_bootstrap(vecm_results, B=1000, alpha=0.05):
    # Given function, simplified for this integration
    return True  # Placeholder for the actual bootstrap logic

vecm_results = []
cointegrated_pairs = []
z_scores_per_pair = []

for pair in pairs:
    ts1, ts2 = returns[pair[0]], returns[pair[1]]
    vector = pd.concat([ts1, ts2], axis=1).dropna()

    if not all(adfuller(vector[col])[1] < significance_level for col in vector.columns):
        continue  # Skip non-stationary pairs

    model_selection = select_order(vector, maxlags=10, deterministic="co")
    lag = model_selection.selected_orders['aic']
    
    if lag < 1:
        continue  # Skip if selected lag is too small

    try:
        vecm = VECM(vector, k_ar_diff=lag-1, coint_rank=1, deterministic='co').fit()
        vecm_test = coint_johansen(vector, det_order=0, k_ar_diff=lag-1)

        if vecm_test.lr1[0] > vecm_test.cvt[0, 1]:  # Simple cointegration check
            if perform_wb and not wild_bootstrap(vecm, B=1000, alpha=significance_level):
                continue  # Skip if bootstrap test fails

        vecm_results.append(vecm)
        cointegrated_pairs.append(pair)

        residuals = vecm.resid
        z_score = (residuals - np.mean(residuals, axis=0)) / np.std(residuals, axis=0)
        z_scores_per_pair.append(z_score.tolist())

    except Exception as e:
        print(f"Error in VECM testing for pair {pair}: {e}")
        continue

print("Number of Cointegrated Pairs:", len(cointegrated_pairs))
for index, pair in enumerate(cointegrated_pairs):
    print(f"Pair #{index}: {pair}")


Number of Cointegrated Pairs: 574
Pair #0: ('EUR', 'GBP')
Pair #1: ('EUR', 'CNY')
Pair #2: ('EUR', 'CHF')
Pair #3: ('EUR', 'HKD')
Pair #4: ('EUR', 'SGD')
Pair #5: ('EUR', 'SEK')
Pair #6: ('EUR', 'NOK')
Pair #7: ('EUR', 'NZD')
Pair #8: ('EUR', 'INR')
Pair #9: ('EUR', 'MXN')
Pair #10: ('EUR', 'TWD')
Pair #11: ('EUR', 'DKK')
Pair #12: ('EUR', 'PLN')
Pair #13: ('EUR', 'THB')
Pair #14: ('EUR', 'ILS')
Pair #15: ('EUR', 'IDR')
Pair #16: ('EUR', 'CZK')
Pair #17: ('EUR', 'AED')
Pair #18: ('EUR', 'TRY')
Pair #19: ('EUR', 'HUF')
Pair #20: ('EUR', 'CLP')
Pair #21: ('EUR', 'SAR')
Pair #22: ('EUR', 'MYR')
Pair #23: ('EUR', 'RUB')
Pair #24: ('EUR', 'RON')
Pair #25: ('EUR', 'PEN')
Pair #26: ('EUR', 'BTC-USD')
Pair #27: ('EUR', 'ETH-USD')
Pair #28: ('EUR', 'LINK-USD')
Pair #29: ('JPY', 'GBP')
Pair #30: ('JPY', 'CHF')
Pair #31: ('JPY', 'HKD')
Pair #32: ('JPY', 'NOK')
Pair #33: ('JPY', 'NZD')
Pair #34: ('JPY', 'TWD')
Pair #35: ('JPY', 'BRL')
Pair #36: ('JPY', 'PLN')
Pair #37: ('JPY', 'THB')
Pair #38: ('J

In [36]:
traded_pairs = []
for pair in cointegrated_pairs:
    pair_index = cointegrated_pairs.index(pair)
    traded_pairs.append(pair_index)


z_scores_matrix = np.vstack(z_scores_per_pair)
print(z_scores_matrix)


[[ 0.92934894  0.13840585]
 [-0.05196909 -0.12971546]
 [ 0.13622416  0.95876537]
 ...
 [-0.06417902  0.13979458]
 [ 0.33676355 -0.16067408]
 [ 0.47706119  1.04827108]]


In [37]:
transaction_costs = np.arange(0, 0.0100, 0.0001)


In [39]:
def balance_function(z_scores, rates_1, rates_2, beta, pair, transaction_costs, 
                      ):
    results = []
    open_threshold = 1.25
    close_threshold = 0.5
    stoploss_threshold = 3.75
    
    for cost in range(len(transaction_costs)):
        if len(rates_1) != len(z_scores) or len(rates_2) != len(z_scores):
            raise ValueError("Lengths of rates and z_scores must be equal.")
        
        cum_return = 1
        stock_status = 0  # 0 is untraded, 1 is long and -1 is short
        trade_log = []  # Initialize trade log for each cost
        trade_log.append(f"\n--- Transaction cost of {transaction_costs[cost] * 100}% ---")
        trade_log.append(f"Trades made on pair no. {pair}:")
        temp_results = []
        
        for i in range(len(z_scores)):
            current_z = z_scores[i]
            
            if stock_status == 0:
                if -open_threshold < current_z < -stoploss_threshold:
                    open_in = beta * rates_2[i] * (1 - transaction_costs[cost])
                    open_out = rates_1[i] * (1 + transaction_costs[cost])
                    stock_status = 1
                    trade_log.append(f"Day {i}: Long position opened on {pair}")
                
                if open_threshold < current_z < stoploss_threshold:
                    open_out = beta * rates_2[i] * (1 + transaction_costs[cost])
                    open_in = rates_1[i] * (1 - transaction_costs[cost])
                    stock_status = -1
                    trade_log.append(f"Day {i}: Short position opened on {pair}")
            
            if abs(current_z) > stoploss_threshold and stock_status != 0:
                if stock_status == 1:
                    close_in = rates_1[i] * (1 - transaction_costs[cost])
                    close_out = beta * rates_2[i] * (1 + transaction_costs[cost])
                    stock_status = 0
                    trade_log.append(f"Day {i}: Long position closed (stop loss) on {pair}")
                
                if stock_status == -1:
                    close_out = rates_1[i] * (1 + transaction_costs[cost])
                    close_in = beta * rates_2[i] * (1 - transaction_costs[cost])
                    stock_status = 0
                    trade_log.append(f"Day {i}: Short position closed (stop loss) on {pair}")
                
                ret = (close_in + open_in) / (close_out + open_out) - 1
                cum_return += ret
                temp_results.append(ret)
                trade_log.append(f"At return of {ret * 100}%")
                continue
            
            if -close_threshold < current_z and stock_status == 1:
                close_in = rates_1[i] * (1 - transaction_costs[cost])
                close_out = beta * rates_2[i] * (1 + transaction_costs[cost])
                ret = (close_in + open_in) / (close_out + open_out) - 1
                cum_return += ret
                temp_results.append(ret)
                stock_status = 0
                trade_log.append(f"Day {i}: Long position closed on {pair} at return of {ret * 100}%")
                
                if current_z > open_threshold:
                    open_out = beta * rates_2[i] * (1 + transaction_costs[cost])
                    open_in = rates_1[i] * (1 - transaction_costs[cost])
                    stock_status = -1
                    trade_log.append(f"Day {i}: Short position opened on {pair}")
                continue
            
            if current_z < close_threshold and stock_status == -1:
                close_out = rates_1[i] * (1 + transaction_costs[cost])
                close_in = beta * rates_2[i] * (1 - transaction_costs[cost])
                ret = (close_in + open_in) / (close_out + open_out) - 1
                cum_return += ret
                temp_results.append(ret)
                stock_status = 0
                trade_log.append(f"Day {i}: Short position closed on {pair} at return of {ret * 100}%")
                
                if current_z < -open_threshold:
                    open_out = rates_1[i] * (1 + transaction_costs[cost])
                    open_in = beta * rates_2[i] * (1 - transaction_costs[cost])
                    stock_status = 1
                    trade_log.append(f"Day {i}: Long position opened on {pair}")
                continue
            
            temp_results.append(0)
        
        if stock_status == 1:
            close_in = rates_1[-1] * (1 - transaction_costs[cost])
            close_out = beta * rates_2[-1] * (1 + transaction_costs[cost])
            ret = (close_in + open_in) / (close_out + open_out) - 1
            cum_return += ret
            temp_results.append(ret)
            trade_log.append(f"Final day: Long position closed on {pair} at return of {ret * 100}%")
        
        if stock_status == -1:
            close_in = beta * rates_2[-1] * (1 - transaction_costs[cost])
            close_out = rates_1[-1] * (1 + transaction_costs[cost])
            ret = (close_in + open_in) / (close_out + open_out) - 1
            cum_return += ret
            temp_results.append(ret)
            trade_log.append(f"Final day: Short position closed on {pair} at return of {ret * 100}%")
        
        results.append(temp_results)
        trade_log.append(f"Final return on pair {pair}: {(cum_return - 1) * 100}%")
    
    return results, trade_log


In [40]:
def run_cum_returns(trade_returns):
    cum_returns = [1]  # Start with initial investment
    for ret in trade_returns:
        cum_returns.append(cum_returns[-1] * (1 + ret))
    return cum_returns


In [41]:
import numpy as np
import pandas as pd

# Assuming 'rates' and 'returns' DataFrames are already loaded and preprocessed
# And assuming 'vecm_models', 'traded_pairs', and 'cointegrated_pairs' are already defined

# Define the transaction costs as a numpy array for flexibility in Python
transaction_costs = np.arange(0, 0.0101, 0.0001)

# Placeholder for 'balance_function'
def balance_function(z_scores, rates_1, rates_2, beta, pair, transaction_costs):
    # Implementation of your trading logic here
    # For now, return mock results and a trade log
    return [0.01, 0.02, 0.03], ["Trade opened", "Trade closed"]

# Define 'run_cum_returns' in Python
def run_cum_returns(trade_returns):
    cum_returns = [1]  # Start with initial investment
    for ret in trade_returns:
        cum_returns.append(cum_returns[-1] * (1 + ret))
    return cum_returns

# Prepare for backtesting - assuming the last 20% of the data is for backtesting
split_index = int(0.8 * len(rates))
rates_backtest = rates.iloc[split_index:]

results = []
cumulative_returns = []
filtered_returns = []

for i in traded_pairs:
    pair_index = traded_pairs.index(i)
    pair = cointegrated_pairs[i]
    z_scores = z_scores_matrix[pair_index]
    
    # Extracting rates for the pair for backtesting period
    rates_1 = rates_backtest[pair[0]].values
    rates_2 = rates_backtest[pair[1]].values
    
    # Assuming vecm_models[pair_index] contains the beta value
    beta = vecm_results[pair_index].beta[1, 0]
    
    # Execute trading strategy
    pair_results, trade_log = balance_function(z_scores, rates_1, rates_2, beta, i, transaction_costs)
    
    # Calculate cumulative returns for the pair
    pair_cumulative_returns = run_cum_returns(pair_results)
    
    # Filter out zero returns
    pair_filtered_returns = [x for x in pair_results if x != 0]
    
    # Storing results
    results.append(pair_results)
    cumulative_returns.append(pair_cumulative_returns)
    filtered_returns.append(pair_filtered_returns)

# Now 'results', 'cumulative_returns', and 'filtered_returns' contain the trading outcomes


In [42]:
def sum_across_pairs(returns, transaction_costs, traded_pairs):
    total_returns = []

    for i in range(len(transaction_costs)):
        temp_val = []
        for n in traded_pairs:
            # Check if 'i' is within bounds for the nth pair's returns list
            if i < len(returns[n]):
                val = returns[n][i]
                temp_val.append(val)
            else:
                print(f"Transaction cost index {i} out of range for pair index {n}.")
        total_returns.append(sum(temp_val))
    
    return total_returns


In [43]:
def sum_across_time(returns, transaction_costs, traded_pairs):
    # Assuming 'returns' is structured as [pair1_returns, pair2_returns, ...]
    # where each pairX_returns is a list of cumulative returns for different transaction cost scenarios.
    # 'transaction_costs' and 'traded_pairs' are lists that index into 'returns'.
    
    # Initialize a list to store the sum of returns for each transaction cost scenario
    sum_returns_by_scenario = [0 for _ in range(len(transaction_costs))]
    
    for pair_index in traded_pairs:  # Assuming 'traded_pairs' indexes into 'returns'
        pair_returns = returns[pair_index]  # Get cumulative returns for the pair
        
        # Sum the returns for each transaction cost scenario
        for i, return_value in enumerate(pair_returns):
            sum_returns_by_scenario[i] += return_value
            
    return sum_returns_by_scenario

In [44]:
def cumulate_running_time(cumulated_across_time):
    """
    Assumes cumulated_across_time is a list of lists, where each sublist
    represents cumulative returns over time for a specific transaction cost scenario.
    """
    # Initialize an empty list to hold the cumulative sum results
    trans_list = []
    
    # Iterate over each transaction cost scenario
    for scenario_returns in cumulated_across_time:
        if not isinstance(scenario_returns, list):
            # If we encounter a scenario that is not a list, wrap it in a list
            scenario_returns = [scenario_returns]
        
        # Calculate the cumulative sum for the scenario
        cumulative_sum = []
        running_total = 0
        for return_value in scenario_returns:
            running_total += return_value
            cumulative_sum.append(running_total)
        
        # Append the cumulative sum for the scenario to the result list
        trans_list.append(cumulative_sum)
    
    return trans_list


In [45]:
# Ensure 'cumulative_returns', 'transaction_costs', and 'traded_pairs' are correctly defined before this step
total_returns_across_pairs = sum_across_pairs(cumulative_returns, transaction_costs, traded_pairs)
# Ensure 'results' is correctly defined before this step
total_returns_across_time = sum_across_time(results, transaction_costs, traded_pairs)
# Use 'total_returns_across_time' from the previous step
total_returns_running = cumulate_running_time(total_returns_across_time)


Transaction cost index 4 out of range for pair index 0.
Transaction cost index 4 out of range for pair index 1.
Transaction cost index 4 out of range for pair index 2.
Transaction cost index 4 out of range for pair index 3.
Transaction cost index 4 out of range for pair index 4.
Transaction cost index 4 out of range for pair index 5.
Transaction cost index 4 out of range for pair index 6.
Transaction cost index 4 out of range for pair index 7.
Transaction cost index 4 out of range for pair index 8.
Transaction cost index 4 out of range for pair index 9.
Transaction cost index 4 out of range for pair index 10.
Transaction cost index 4 out of range for pair index 11.
Transaction cost index 4 out of range for pair index 12.
Transaction cost index 4 out of range for pair index 13.
Transaction cost index 4 out of range for pair index 14.
Transaction cost index 4 out of range for pair index 15.
Transaction cost index 4 out of range for pair index 16.
Transaction cost index 4 out of range for

In [46]:
print("Length of cumulative_returns:", len(cumulative_returns))
print("Length of transaction_costs:", len(transaction_costs))
for i, returns_pairs in enumerate(cumulative_returns):
    print(f"Length of returns for pair {i}: {len(returns)}")


Length of cumulative_returns: 574
Length of transaction_costs: 101
Length of returns for pair 0: 1255
Length of returns for pair 1: 1255
Length of returns for pair 2: 1255
Length of returns for pair 3: 1255
Length of returns for pair 4: 1255
Length of returns for pair 5: 1255
Length of returns for pair 6: 1255
Length of returns for pair 7: 1255
Length of returns for pair 8: 1255
Length of returns for pair 9: 1255
Length of returns for pair 10: 1255
Length of returns for pair 11: 1255
Length of returns for pair 12: 1255
Length of returns for pair 13: 1255
Length of returns for pair 14: 1255
Length of returns for pair 15: 1255
Length of returns for pair 16: 1255
Length of returns for pair 17: 1255
Length of returns for pair 18: 1255
Length of returns for pair 19: 1255
Length of returns for pair 20: 1255
Length of returns for pair 21: 1255
Length of returns for pair 22: 1255
Length of returns for pair 23: 1255
Length of returns for pair 24: 1255
Length of returns for pair 25: 1255
Length 

In [47]:
for pair_index in traded_pairs:
    pair = cointegrated_pairs[pair_index]
    print(f"Traded Pair #{pair_index}: {pair[0]} and {pair[1]}")

Traded Pair #0: EUR and GBP
Traded Pair #1: EUR and CNY
Traded Pair #2: EUR and CHF
Traded Pair #3: EUR and HKD
Traded Pair #4: EUR and SGD
Traded Pair #5: EUR and SEK
Traded Pair #6: EUR and NOK
Traded Pair #7: EUR and NZD
Traded Pair #8: EUR and INR
Traded Pair #9: EUR and MXN
Traded Pair #10: EUR and TWD
Traded Pair #11: EUR and DKK
Traded Pair #12: EUR and PLN
Traded Pair #13: EUR and THB
Traded Pair #14: EUR and ILS
Traded Pair #15: EUR and IDR
Traded Pair #16: EUR and CZK
Traded Pair #17: EUR and AED
Traded Pair #18: EUR and TRY
Traded Pair #19: EUR and HUF
Traded Pair #20: EUR and CLP
Traded Pair #21: EUR and SAR
Traded Pair #22: EUR and MYR
Traded Pair #23: EUR and RUB
Traded Pair #24: EUR and RON
Traded Pair #25: EUR and PEN
Traded Pair #26: EUR and BTC-USD
Traded Pair #27: EUR and ETH-USD
Traded Pair #28: EUR and LINK-USD
Traded Pair #29: JPY and GBP
Traded Pair #30: JPY and CHF
Traded Pair #31: JPY and HKD
Traded Pair #32: JPY and NOK
Traded Pair #33: JPY and NZD
Traded Pair

In [48]:
for index in traded_pairs:
    # Retrieve the currency pair and its cumulative returns
    currency_pair = cointegrated_pairs[index]
    cumret = cumulative_returns[index]

In [49]:

print(cumulative_returns)

[[1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0302, 1.061106], [1, 1.01, 1.0